In [1]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from catboost import CatBoostClassifier, Pool

In [2]:
''' Data Load '''
train = pd.read_csv("/home/workspace/DACON/Click_predict/data/train3.csv")
test = pd.read_csv("/home/workspace/DACON/Click_predict/data/test3.csv")

In [4]:
feature_summary(train)

---------------------------- Dataframe Size ----------------------------
num: 28605391
----------------------------   Dtype Size   ----------------------------
Type float64 : 20
Type object : 18
Type int64 : 3


,Feature Name,Data Type,Nunique,NullValue,NullValue Ratio,value_1,value_2,value_3
0,ID,object,28605391,0,0.000000,TRAIN_00000000,TRAIN_00000001,TRAIN_00000002
1,Click,int64,2,0,0.000000,1,0,0
2,F01,object,4760930,1234711,4.316358,NSLHFNS,VGIVWZQ,JCDXFYU
3,F02,object,304404,1234711,4.316358,AVKQTCL,LSUSMVO,PILDDJU
4,F04,float64,9978,5742331,20.074296,114.0,26.0,119.0
5,F05,object,5343556,1234711,4.316358,ISVXFVA,NFRVLWS,LFPUEOV
6,F06,int64,8562,0,0.000000,1,43,0
7,F07,object,151200,0,0.000000,PQZBVMG,IMPIGJT,FFUTIRZ
8,F09,object,27551,0,0.000000,IZYJZDA,NGODWIN,BEZTQIO
9,F10,object,1404254,1234711,4.316358,RANQNXO,GBQMJYF,YLKUVQA


In [6]:
categorical_cols = train.select_dtypes(include = "object").columns

# 각 범주형 변수의 빈도수 계산 및 인코딩
for col in tqdm(categorical_cols):
    freq_encoding = train[col].value_counts(normalize=True)
    train[col] = train[col].map(freq_encoding)
    test[col] = test[col].map(freq_encoding)

# 결과 확인
print(train.head())

100%|██████████| 18/18 [01:48<00:00,  6.02s/it]

             ID  Click           F01       F02    F04           F05  F06  \
0  3.495845e-08      1  3.653545e-08  0.000002  114.0  3.653545e-08    1   
1  3.495845e-08      0  4.128505e-05  0.000215   26.0  4.128505e-05   43   
2  3.495845e-08      0  2.147638e-01  0.215404  119.0  2.147638e-01    0   
3  3.495845e-08      1  5.407246e-04  0.005296   15.0  5.407246e-04   26   
4  3.495845e-08      0  3.288190e-07  0.001369   13.0  3.288190e-07   20   

        F07       F09           F10    F11           F12       F13  F14  \
0  0.188244  0.003238  2.557481e-07   66.0  3.653545e-08  0.035374    4   
1  0.000290  0.007269  4.267340e-05  137.0  4.128505e-05  0.019912   22   
2  0.000240  0.038967  2.147638e-01    NaN  2.147638e-01  0.042380    0   
3  0.000007  0.000116  2.457630e-03   50.0  5.407246e-04  0.002023   20   
4  0.188244  0.000209  6.970963e-05  789.0  2.531906e-05  0.001245   42   

        F16   F18  F19       F20       F22   F24       F25       F26   F27  \
0  0.000083   

In [7]:
feature_summary(train)

---------------------------- Dataframe Size ----------------------------
num: 28605391
----------------------------   Dtype Size   ----------------------------
Type float64 : 38
Type int64 : 3


,Feature Name,Data Type,Nunique,NullValue,NullValue Ratio,value_1,value_2,value_3
0,ID,float64,1,0,0.000000,3.495845e-08,3.495845e-08,3.495845e-08
1,Click,int64,2,0,0.000000,1.000000e+00,0.000000e+00,0.000000e+00
2,F01,float64,2002,1234711,4.316358,3.653545e-08,4.128505e-05,2.147638e-01
3,F02,float64,2367,1234711,4.316358,1.863308e-06,2.153399e-04,2.154039e-01
4,F04,float64,9978,5742331,20.074296,1.140000e+02,2.600000e+01,1.190000e+02
5,F05,float64,1952,1234711,4.316358,3.653545e-08,4.128505e-05,2.147638e-01
6,F06,int64,8562,0,0.000000,1.000000e+00,4.300000e+01,0.000000e+00
7,F07,float64,3445,0,0.000000,1.882442e-01,2.897706e-04,2.400596e-04
8,F09,float64,3186,0,0.000000,3.238480e-03,7.269364e-03,3.896689e-02
9,F10,float64,2418,1234711,4.316358,2.557481e-07,4.267340e-05,2.147638e-01


## Label Encoding

In [8]:
''' Seed '''
seed_everything(42)

''' Train / Vaild Split'''
X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

param = {'max_depth': 30,
 'num_leaves': 306,
 'subsample': 0.9757365622458185,
 'subsample_freq': 8,
 "n_estimators" : 1000,
 'min_child_samples': 136}

model = lgb.LGBMClassifier(random_state=42, **param)
model.fit(X_train, y_train, eval_metric='AUC')
pred = model.predict_proba(X_valid)
score = roc_auc_score(y_valid, pred[:, 1])
print(score)

0.7801199049542845


## Baseline

In [2]:
''' Data Load '''
train = pd.read_csv("/home/workspace/DACON/Click_predict/data/train3.csv")
test = pd.read_csv("/home/workspace/DACON/Click_predict/data/test3.csv")

categorical = train.select_dtypes(include='object').columns 
train[categorical] = train[categorical].astype("category")
test[categorical] = test[categorical].astype("category")

''' Seed '''
seed_everything(42)

''' Train / Vaild Split'''
X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

param = {'max_depth': 30,
 'num_leaves': 306,
 'subsample': 0.9757365622458185,
 'subsample_freq': 8,
 "n_estimators" : 1000,
 'min_child_samples': 136}

model = lgb.LGBMClassifier(random_state=42, **param)
model.fit(X_train, y_train, eval_metric='AUC')
pred = model.predict_proba(X_valid)
score = roc_auc_score(y_valid, pred[:, 1])
print(score)

0.7801998657255164


## skfold

In [5]:
''' Seed '''
seed_everything(42)

''' Cat '''
categorical = train.select_dtypes(include='object').columns 
train[categorical] = train[categorical].astype("category")
test[categorical] = test[categorical].astype("category")

''' Train / Vaild Split'''
param = {'max_depth': 30,
 'num_leaves': 306,
 'subsample': 0.9757365622458185,
 'subsample_freq': 8,
 "n_estimators" : 1000,
 'min_child_samples': 136}

X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]

''' Model '''
model = lgb.LGBMClassifier(random_state=42, **param)

''' K-Fold '''
valid_array, test_array, voting = Kfold(model, 5, X_train, y_train, test, True)

: 

In [4]:
X_train.head()

,ID,F01,F02,F04,F05,F06,F07,F09,F10,F11,F12,F13,F14,F16,F18,F19,F20,F22,F24,F25,F26,F27,F29,F30,F32,F33,F34,F36,F37,F38,F39,F03,F08,F15,F17,F21,F23,F28,F31,F35
0,TRAIN_00000000,NSLHFNS,AVKQTCL,114.0,ISVXFVA,1,PQZBVMG,IZYJZDA,RANQNXO,66.0,EGWPZEB,SMRBWMU,4,NLHSWSR,NaN,1.0,LTCDFSX,SNDDHSM,5.0,HLADEES,XAUNDQW,3.0,1.0,NZGEZLW,380.0,2.0,AXQFZWC,NaN,TFJMLCZ,0.0,AURZYDY,0.187347,0.215969,0.223602,0.172690,0.258027,0.194720,0.183423,0.203653,0.2048
1,TRAIN_00000001,VGIVWZQ,LSUSMVO,26.0,NFRVLWS,43,IMPIGJT,NGODWIN,GBQMJYF,137.0,OUPRLWN,QVLCQTS,22,DGLEHCI,19.0,1.0,QAOROGG,SNDDHSM,21.0,TPNEYOY,BUWDIBR,32.0,2.0,NZGEZLW,466.0,1.0,DRVVDHZ,19.0,AUGTURV,0.0,LUZRMLU,0.187075,0.234439,0.180882,0.249527,0.216504,0.194720,0.234148,0.203653,0.2048
2,TRAIN_00000002,JCDXFYU,PILDDJU,119.0,LFPUEOV,0,FFUTIRZ,BEZTQIO,YLKUVQA,NaN,MGRUYII,KGJACUM,0,PBCDAMR,2.0,0.0,TGVLSYT,SNDDHSM,NaN,YMGUAHK,WVRHGBC,NaN,NaN,VHXETCF,197.0,0.0,QMOULXS,8.0,ZVSTLNM,0.0,MHBRSQK,0.169448,0.141576,0.167974,0.147666,0.133926,0.194653,0.139434,0.151770,0.2048
3,TRAIN_00000003,PSMFWTP,ZYAVJHP,15.0,ATQPZSJ,26,ZDTZNSB,LTETYBG,GEKHGQZ,50.0,OPGPVBH,UMIEGWH,20,WDTKLCQ,14.0,0.0,NaN,SNDDHSM,87.0,IROGIOW,NaN,2.0,1.0,IVIRTPR,8640.0,0.0,IZLJUJS,14.0,ZBSRLCQ,0.0,GAZBSSZ,0.218804,0.206054,0.218804,0.172690,0.172939,0.194753,0.206054,0.203653,0.2048
4,TRAIN_00000004,SLCRICD,QPQWGXA,13.0,CHZGJZR,20,PQZBVMG,LJBQPJW,SOKLCDW,789.0,LHVQODU,YJFFIGH,42,NRRIGTN,13.0,0.0,NaN,NXBZHKD,NaN,PWGXQTB,NaN,NaN,NaN,NZGEZLW,41774.0,0.0,BHBIZCL,13.0,QHYLSBX,0.0,QTATWAY,0.218804,0.234439,0.218804,0.147666,0.162883,0.195566,0.234148,0.079718,0.2048
